In [1]:
from dataset import (
    get_dataset, 
    save_map, 
    save_pred_map_to_tif,
    save_dataset_to_tif,
)
from metrics import (
    get_metrics,
    get_classification_metrics,
    get_feature_importances, 
    get_catboost_importances, 
)
from vis import (
    plot_image,
    plot_mask,
    plot_SCL,
    draw_maps,
)

import matplotlib.pyplot as plt
from rasterio.windows import Window
import pandas as pd
import wandb
import rasterio
import numpy as np
import tifffile as tiff
import wandb
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2   

/beegfs/home/alina.smolina/forest-co2/dataset.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import rasterio
from rasterio.windows import Window
from shapely.geometry.polygon import Polygon
from shapely import geometry
import tifffile as tiff
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from pathlib import Path
import uuid

from dataset import (
    get_dataset, 
    save_map,
)
from vis import (
    plot_res
)

%load_ext autoreload
%autoreload 2   

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# path_S2B_list = [ 
    # './data/tile_UTM38N/20180804/',
    # './data/tile_UTM38N/20180827/',
    # './data/tile_UTM38N/20180911/',
    # './data/tile_UTM38N/20190608/',
    # './data/tile_UTM38N/20190613/',
    # './data/tile_UTM38N/20200604/',
    # './data/tile_UTM38N/20200709/',
# ]
path_S2B = './data/tile_UTM38N/20180730/'

channels = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12']

In [4]:
# Get table with types and ages
y0, x0 = 6000, 0
window_n = Window.from_slices((y0, y0 + 4000), (x0, x0 + 2700))

y0, x0 = 0, 5700
window_r = Window.from_slices((y0, y0 + 7000), (x0, x0 + 10000))


### Test
y0, x0 = 7500, 2700
window_k = Window.from_slices((y0, y0 + 1952), (x0, x0 + 1200))

In [5]:
from catboost import CatBoostClassifier, CatBoostRegressor
model_species = CatBoostClassifier()
model_species.load_model('weights/species-10bands-indices-10000')

In [6]:
model_age = CatBoostRegressor()
model_age.load_model('weights/age-10bands-indices')

In [7]:
model_height = CatBoostRegressor()
model_height.load_model('weights/height-10bands-indices')

In [8]:
model_area = CatBoostRegressor()
model_area.load_model('weights/basal-area-10bands-indices')

In [9]:
def predict_volume(X):
    basal_area = model_area.predict(X)
    basal_area = np.where(basal_area > 0, basal_area, 0)
    
    height = model_height.predict(X)
    height = np.where(height > 0, height, 0)
    
    volume = height * basal_area
    volume = np.where((height > 0) & (basal_area > 0), volume, 0)
    
    return volume

In [10]:
def predict_stock(X):
    age = model_age.predict(X)
    age = np.where(age > 0, age, 0)
    
    species = model_species.predict(X)
    
    volume = predict_volume(X)
    
    species_age = {
    1: [[0, 40], [40, 100], [100, 120], [120, 1000]],
    2: [[0, 20], [20, 40], [40, 50], [50, 1000]],
    3: [[0, 20], [20, 40], [40, 50], [50, 1000]],
    4: [[0, 20], [20, 30], [30, 40], [40, 1000]],
    5: [[0, 40], [40, 100], [100, 120], [120, 1000]],
              }

    species_coeff = {
        1: [0.409, 0.426, 0.431, 0.436],
        2: [0.528, 0.534, 0.533, 0.528],
        3: [0.45, 0.46, 0.46, 0.46], #3: [0.4, 0.4, 0.4, 0.4],
        4: [0.403, 0.426, 0.431, 0.444],
        5: [0.452, 0.454, 0.454, 0.449],
                    }

    C_content = {
        1: 0.5,
        2: 0.47,
        3: 0.48,
        4: 0.47, 
        5: 0.5
    }
    
    BCEF_species = {
    1: np.zeros(volume.shape),
    2: np.zeros(volume.shape), 
    3: np.zeros(volume.shape),
    4: np.zeros(volume.shape),
    5: np.zeros(volume.shape),
               }

    tmp_shape = np.ones(volume.shape)

    for sp in BCEF_species.keys():
        for age_ind in range(4):
            specimen_mask = np.where(species == sp, 1, 0) 
            age_condition = (species_age[sp][age_ind][0] * tmp_shape < age) & (age <= species_age[sp][age_ind][1] * tmp_shape)
            BCEF_species[sp] += np.where(age_condition, np.squeeze(specimen_mask * species_coeff[sp][age_ind]), 0)
        BCEF_species[sp] *= C_content[sp]

    carbon_stock = volume * sum(BCEF_species.values())
    carbon_stock = np.where(carbon_stock > 0, carbon_stock, 0)
    
    return carbon_stock

## Сохранить предсказания в tif

In [11]:
dataset = get_dataset(
        path_S2B, 
        channels_list=channels, 
        window=window_k, 
        visualise=False, 
        indices=True
)

X = dataset.drop(labels=['x', 'y'], axis=1)

### Возраст

In [20]:
preds = model_age.predict(X)
save_map(
    preds, 
    dataset[['x', 'y']], 
    window=window_k, 
    no_data_value=-1, 
    clip=(0, None), 
    path_S2B=path_S2B, 
    out_name='preds_age-10bands-indices.tif',
    out_path = './preds/'
)

Saved to preds/preds_age-10bands-indices.tif
Out dtype: float32


### Высота

In [22]:
preds = model_height.predict(X)
save_map(
    preds, 
    dataset, 
    window=window_k, 
    no_data_value=-1, 
    clip=(0, None), 
    path_S2B=path_S2B, 
    out_name='preds_height-10bands-indices.tif',
    out_path = './preds/'
        )

Saved to preds/preds_height-10bands-indices.tif
Out dtype: float32


### Basal area

In [23]:
preds = model_area.predict(X)
save_map(
    preds, 
    dataset, 
    window=window_k, 
    no_data_value=-1, 
    clip=(0, None), 
    path_S2B=path_S2B, 
    out_name='preds_basal-area-10bands-indices.tif',
    out_path = './preds/',
        )

Saved to preds/preds_basal-area-10bands-indices.tif
Out dtype: float32


### Volume 

In [24]:
preds = predict_volume(X)
save_map(
    preds, 
    dataset, 
    window=window_k, 
    no_data_value=-1, 
    clip=(0, None), 
    path_S2B=path_S2B, 
    out_name='preds_volume.tif',
    out_path = './preds/'
        )

Saved to preds/preds_volume.tif
Out dtype: float32


### Carbon stock

In [12]:
preds = predict_stock(X)
save_map(
    preds, 
    dataset, 
    window=window_k, 
    no_data_value=-1, 
    clip=(0, None), 
    path_S2B=path_S2B, 
    out_name='preds_stock.tif',
    out_path = './preds/'
        )

TBB Warning: The number of workers is currently limited to 15. The request for 127 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



Saved to preds/preds_stock.tif
Out dtype: float32


### Species

In [26]:
preds = model_species.predict(X)
save_map(preds, 
         dataset, 
         window=window_k, 
         no_data_value=-1, 
         clip=(None, 10), 
         path_S2B=path_S2B, 
         out_name='preds_species-10bands-indices-10000.tif',
         out_path = './preds/'
        )

Saved to preds/preds_species-10bands-indices-10000.tif
Out dtype: float32


In [27]:
!ls -lah './preds/'

total 1.4M
drwxrwxr-x  2 alina.smolina alina.smolina    6 Dec  8 16:33 .
drwxrwxr-x 14 alina.smolina alina.smolina   33 Dec  8 16:32 ..
-rw-rw-r--  1 alina.smolina alina.smolina 9.0M Dec  8 16:31 preds_age-10bands-indices_DEL.tif
-rw-rw-r--  1 alina.smolina alina.smolina 9.0M Dec  8 16:32 preds_age-10bands-indices.tif
-rw-rw-r--  1 alina.smolina alina.smolina 9.0M Dec  8 16:32 preds_basal-area-10bands-indices.tif
-rw-rw-r--  1 alina.smolina alina.smolina 9.0M Dec  8 16:32 preds_height-10bands-indices.tif
-rw-rw-r--  1 alina.smolina alina.smolina 9.0M Dec  8 16:33 preds_species-10bands-indices-10000.tif
-rw-rw-r--  1 alina.smolina alina.smolina 9.0M Dec  8 16:33 preds_stock.tif
-rw-rw-r--  1 alina.smolina alina.smolina 9.0M Dec  8 16:32 preds_volume.tif
